In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

def make_layer(in_feature, out_feature):
    layer = nn.Sequential(*[
            nn.Linear(in_feature, out_feature),
            nn.BatchNorm1d(out_feature),
            nn.ReLU()
        ])
    return layer

class Hardswish(nn.Module):
    def __init__(self, inplace=False):
        super(Hardswish, self).__init__()

    def forward(self, x):
        return (x > 3).to(dtype=torch.float) * x + (x > -3).to(dtype=torch.float) * (x < 3).to(dtype=torch.float) * (x*(x+3)/6)

class BottleneckEncoder(nn.Module):
    def __init__(self, in_feature, dev):
        super(BottleneckEncoder, self).__init__()
        self.dev = dev
        self.in_bn = nn.BatchNorm1d(in_feature)
        self.fc1 = nn.Linear(in_feature, 640)
        self.act1 = nn.ReLU(inplace=True)
        # self.bn1 = nn.BatchNorm1d(128)
        self.dp1 = nn.Dropout(p=0.2)

        self.out_fc = nn.Linear(640, in_feature)
        # self.fc2 = nn.Linear(128, 64)
        # self.act2 = nn.ReLU(inplace=True)
        # self.bn2 = nn.BatchNorm1d(64)
        # self.dp2 = nn.Dropout(p=0.2)

        # self.fc3 = nn.Linear(64, 128)
        # self.act3 = nn.ReLU(inplace=True)
        # self.bn3 = nn.BatchNorm1d(128)
        # self.dp3 = nn.Dropout(p=0.2)

        # self.out_fc = nn.Linear(128, in_feature)

    def forward(self, x):
        x = self.in_bn(x)
        x = x + self.dev * torch.randn(*x.size()).to(x.device)
        x = self.dp1(self.act1(self.fc1(x)))

        return self.out_fc(x), x

class AEClassifier(nn.Module):
    def __init__(self, in_feature, out_feature):
        super(AEClassifier, self).__init__()
        self.fc1 = nn.Linear(in_feature, 320)
        self.act1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(320)
        self.dp1 = nn.Dropout(p=0.2)

        self.out_fc = nn.Linear(320, out_feature)

    def forward(self, x):
        x = self.dp1(self.bn1(self.act1(self.fc1(x))))

        return torch.sigmoid(self.out_fc(x))



class MLPClassifier(nn.Module):
    def __init__(self, in_feature, out_feature):
        super(MLPClassifier, self).__init__()
        self.in_bn = nn.BatchNorm1d(in_feature)
        self.in_dp = nn.Dropout(p=0.2)

        self.fc1 = nn.Linear(in_feature, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.act1 = Hardswish(inplace=True)
        self.dp1 = nn.Dropout(p=0.2)

        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.act2 = Hardswish(inplace=True)
        self.dp2 = nn.Dropout(p=0.2)

        self.fc3 = nn.Linear(128, 64)
        self.bn3 = nn.BatchNorm1d(64)
        self.act3 = Hardswish(inplace=True)
        self.dp3 = nn.Dropout(p=0.2)

        self.out_fc = nn.Linear(64, out_feature)

    def forward(self, x):
        x = self.in_dp(self.in_bn(x))
        x = self.dp1(self.act1(self.bn1(self.fc1(x))))

        x = self.dp2(self.act2(self.bn2(self.fc2(x))))

        x = self.dp3(self.act3(self.bn3(self.fc3(x))))
        return torch.sigmoid(self.out_fc(x))
class MLP(nn.Module):
    '''
    layer_setting[list]: [130,64,2]
    '''
    def __init__(self, layer_setting=[130,64,2]):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(*[make_layer(layer_setting[i], layer_setting[i+1]) for i in range(len(layer_setting)-2)])
        self.out_layer = nn.Linear(layer_setting[-2], layer_setting[-1])

    def forward(self, x):
        
        return torch.sigmoid(self.out_layer(self.layers(x)))

class AutoEncoder(nn.Module):
    def __init__(self, layer_setting):
        super(AutoEncoder, self).__init__()
        self.layers = nn.ModuleList([make_layer(layer_setting[i], layer_setting[i+1]) for i in range(len(layer_setting)-2)])
        self.out_layer = nn.Linear(layer_setting[-2], layer_setting[-1])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.out_layer(x)


In [ ]:
TRAINING = False
gpu_device = torch.device('cuda:0')
feat_tags = ['feature_{}'.format(i) for i in range(130)]
resp_tags = ['resp'] + ['resp_{}'.format(i) for i in range(1,5)]
epsilons = [0] * len(resp_tags)
num_epoch = 1
lr = 1e-3
batch_size = 4096
loader_num_workers = 8
print_every = 100
save_every = 1000


FOLDS = 5

train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
train = train.query('date > 85').reset_index(drop = True) 
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)
train = train.query('weight > 0').reset_index(drop = True)
#train['action'] = (train['resp'] > 0).astype('int')
train['action'] =  (  (train['resp_1'] > 0.00001 ) & (train['resp_2'] > 0.00001 ) & (train['resp_3'] > 0.00001 ) & (train['resp_4'] > 0.00001 ) &  (train['resp'] > 0.00001 )   ).astype('int')
f_mean = np.mean(train[feat_tags[1:]].values,axis=0)
print(train.columns)
print('Whole Shape {}'.format(train.shape))

In [ ]:

in_feat = len(feat_tags)
out_feat = len(resp_tags)
dev = 0.1
FOLDS = 5
ae = BottleneckEncoder(in_feat, dev) 
# Final Classifier
cls_models = [MLPClassifier(640+in_feat, out_feat) for i in range(FOLDS)]

restore_path = '../input/js-bottleneck/5_7_half_ae_ckpt.pth.tar'
if os.path.exists(restore_path):
    # restore_path = self.args.restore_checkpoint_path
    print('Restoring from checkpoint:{}'.format(restore_path))
    checkpoint = torch.load(restore_path, map_location=gpu_device)
    #     if 'net_classifier_state' in checkpoint and checkpoint['net_classifier_state'] is not None:
    #         classifier.load_state_dict(checkpoint['net_classifier_state'])
    #         print('Loading Classifier.')
    if 'net_ae_state' in checkpoint and checkpoint['net_ae_state'] is not None:
        ae.load_state_dict(checkpoint['net_ae_state'])
        print('Loading AutoEncoder.')
        
models_restore_path = '../input/js-bottleneck/5_7_half_models_ckpt.pth.tar'
if os.path.exists(models_restore_path):
    checkpoint = torch.load(models_restore_path, map_location=gpu_device)
    for i, cls_model in enumerate(cls_models):
        cls_model.load_state_dict(checkpoint[i])
        cls_model.eval()
        print('Loading Classifiers.')
ae.eval()


cls_models = [cls_models[i] for i in [0,2,3,4]]
import janestreet
# janestreet.competition.make_env.__called__ = False
env = janestreet.make_env()
th = 0.5
#w = np.asarray([0.1,0.1,0.1,0.5,0.2])
for (test_df, pred_df) in env.iter_test():
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, feat_tags].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        x_tt = torch.from_numpy(x_tt).to(dtype=torch.float)
        x_recon, x_enc = ae(x_tt)
        x_con = torch.cat([x_tt, x_enc], dim=1)
        pred = torch.cat([cls_model(x_con) for cls_model in cls_models], dim=1) #classifier(x_con)
        pred_df.action = int((pred.mean(dim=1) > th).to(dtype=torch.float).item())
    else:
        pred_df.action = 0
    env.predict(pred_df)